Following tutorial at <https://python.langchain.com/docs/integrations/graphs/ontotext/>.

In [27]:
from langchain_community.graphs import OntotextGraphDBGraph
#from langchain_openai import ChatOpenAI
import os
from langchain.chains import OntotextGraphDBQAChain
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
graph = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/coptic-metadata-viewer",
    local_file="/Users/sjhuskey/Python/coptic_metadata_viewer/data/coptic-metadata-viewer.ttl",
)

In [ ]:
GRAPHDB_SPARQL_GENERATION_TEMPLATE = """
  You are an expert in SPARQL queries and RDF graph structures. Your task is to generate a SPARQL query based on the provided schema and the user's question.
  Use only the node types and properties provided in the schema.
  Do not use any node types or properties not explicitly listed.
  Include all necessary PREFIX declarations.
  Return only the SPARQL query.
  Use a (shorthand for rdf:type) to declare the class of a subject.
  Do not wrap the query in backticks.
  Do not use triple backticks or any markdown formatting.
  Do not include any text except the SPARQL query generated.
  Always return a human-readable label instead of a URI when possible.
  Do not use multiple WHERE clauses.
  
  Your RDF graph has these prefixes, classes, and properties:

    @prefix coptic: <http://www.semanticweb.org/sjhuskey/ontologies/2025/7/coptic-metadata-viewer/> .
    @prefix dcmitype: <http://purl.org/dc/dcmitype/> .
    @prefix dcterms: <http://purl.org/dc/terms/> .
    @prefix foaf: <http://xmlns.com/foaf/0.1/> .
    @prefix frbr: <http://purl.org/vocab/frbr/core#> .
    @prefix lawd: <http://lawd.info/ontology/> .
    @prefix ns1: <http://lexinfo.net/ontology/2.0/lexinfo#> .
    @prefix owl: <http://www.w3.org/2002/07/owl#> .
    @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix schema1: <http://schema.org/> .
    @prefix skos: <http://www.w3.org/2004/02/skos/core#> .
    @prefix time: <http://www.w3.org/2006/time#> .
    
    - coptic:Colophon:
      - dcterms:description
      - dcterms:identifier
      - dcterms:isPartOf
      - dcterms:references
      - dcterms:type
      - ns1:translation
      - rdf:type
      - time:hasBeginning
      - time:hasEnd

    - coptic:Title:
      - dcterms:description
      - dcterms:identifier
      - dcterms:isPartOf
      - dcterms:references
      - dcterms:type
      - rdf:type

    - dcmitype:Collection:
      - dcterms:hasPart
      - dcterms:identifier
      - dcterms:spatial
      - rdf:type

    - dcterms:Agent:
      - dcterms:creator
      - dcterms:description
      - dcterms:identifier
      - foaf:name
      - owl:sameAs
      - rdf:type
      - schema1:title

    - dcterms:PhysicalResource:
      - dcterms:bibliographicCitation
      - dcterms:description
      - dcterms:hasPart
      - dcterms:identifier
      - dcterms:isPartOf
      - dcterms:medium
      - rdf:type
      - time:hasBeginning
      - time:hasEnd

    - foaf:Person:
      - dcterms:identifier
      - dcterms:isReferencedBy
      - foaf:name
      - ns1:transliteration
      - rdf:type
      - rdfs:label
      - schema1:birthPlace
      - schema1:gender
      - schema1:roleName
      - time:hasBeginning
      - time:hasEnd

    - frbr:Work:
      - dcterms:creator
      - dcterms:description
      - dcterms:identifier
      - dcterms:isPartOf
      - dcterms:isReferencedBy
      - dcterms:temporal
      - dcterms:title
      - rdf:type
      - rdfs:label

    - lawd:Place:
      - lawd:primaryForm
      - rdf:type
      - rdfs:label
      - skos:exactMatch
    
    Schema: {schema}
  
    The question delimited by triple backticks is:
  ```
  {prompt}
  ```
  """
GRAPHDB_SPARQL_GENERATION_PROMPT = PromptTemplate(
      input_variables=["schema", "prompt"],
      template=GRAPHDB_SPARQL_GENERATION_TEMPLATE,
  )

In [30]:
GRAPHDB_QA_TEMPLATE = """Task: Generate a natural language response from the results of a SPARQL query.
  You are an assistant that creates well-written and human understandable answers.
  The information part contains the information provided, which you can use to construct an answer.
  The information provided is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
  Make your response sound like the information is coming from an AI assistant, but don't add any information.
  Don't use internal knowledge to answer the question, just say you don't know if no information is available.
  Information:
  {context}
  
  Question: {prompt}
  Helpful Answer:"""
GRAPHDB_QA_PROMPT = PromptTemplate(
      input_variables=["context", "prompt"], template=GRAPHDB_QA_TEMPLATE
  )

In [31]:
GRAPHDB_SPARQL_FIX_TEMPLATE = """
  This following SPARQL query delimited by triple backticks
  ```
  {generated_sparql}
  ```
  is not valid.
  The error delimited by triple backticks is
  ```
  {error_message}
  ```
  - Do NOT include any Markdown formatting (e.g. no ```sparql or backticks).
  - Do NOT output explanations, only the corrected query.
  - Always start with any necessary PREFIX declarations.
  - Use `a` instead of `rdf:type` to state class membership.
  - Ensure that classes like `frbr:Work` are used as objects, not predicates.
  - Fix common mistakes like using classes as predicates, missing semicolons, or malformed FILTER clauses.

  Only output a valid, working SPARQL query.
  ```
  {schema}
  ```
  """
  
GRAPHDB_SPARQL_FIX_PROMPT = PromptTemplate(
      input_variables=["error_message", "generated_sparql", "schema"],
      template=GRAPHDB_SPARQL_FIX_TEMPLATE,
  )

In [32]:
chain = OntotextGraphDBQAChain.from_llm(
    #ChatOpenAI(model="gpt-4o-mini", temperature=0),
    OllamaLLM(model="mistral-small3.2", temperature=0),
    graph=graph,
    sparql_generation_prompt=GRAPHDB_SPARQL_GENERATION_PROMPT,
    qa_prompt=GRAPHDB_QA_PROMPT,
    sparql_fix_prompt=GRAPHDB_SPARQL_FIX_PROMPT,
    max_fix_retries=3,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
# chain.invoke({chain.input_key: "Who is the creator of the work with the title 'Sermo asceticus?'"})[chain.output_key]
output = chain.invoke("Who wrote the work with the title 'Sermo asceticus'?")  # Example query
print(output['result'])  # Print the answer



> Entering new OntotextGraphDBQAChain chain...


In [ ]:
# chain.invoke({chain.input_key: "Who is the creator of the work with the title 'Sermo asceticus?'"})[chain.output_key]
output = chain.invoke("Which manuscript contains a title with identifier 307?")  # Example query
print(output['result'])  # Print the answer



> Entering new OntotextGraphDBQAChain chain...
Invalid SPARQL query: 
```
PREFIX coptic: <http://www.semanticweb.org/sjhuskey/ontologies/2025/7/coptic-metadata-viewer/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>

SELECT ?manuscriptLabel WHERE {
  ?title a coptic:Title ;
         dcterms:identifier "307" ;
         dcterms:isPartOf ?manuscript .
  ?manuscript rdfs:label ?manuscriptLabel .
}
```
SPARQL Query Parse Error: 
Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '`'  (at char 0), (line:1, col:1)

Generated SPARQL:
PREFIX coptic: <http://www.semanticweb.org/sjhuskey/ontologies/2025/7/coptic-metadata-viewer/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?manuscriptLabel WHERE {
  ?title a coptic:Title ;
         dcterms:identifier "307" ;
         dcterms:isPartOf ?manuscript .
  ?manuscript rdfs:label ?manuscriptLabel .
}

> Finished chain.
I don'